In [1]:
# All  import statements needed for the notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.metrics import *

In [2]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_csv('/content/drive/MyDrive/balanced_data.csv')

In [5]:
data.drop(data.columns[0] , inplace=True , axis=1)

In [6]:
unknown = ['SSH-Patator','DoS slowloris','DoS Slowhttptest','Bot','Infiltration','Heartbleed']
att = data.loc[(data['Label'].isin(unknown))]

In [7]:
# importing required libraries for normalizing data
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

In [8]:
# selecting numeric attributes columns from data
numeric_col = data.select_dtypes(include='number').columns

In [9]:
# using standard scaler for normalizing
std_scaler = MinMaxScaler()
def normalization(df,att,col):
  for i in col:
    arr = df[i]
    arr = np.array(arr)
    x = np.array(att[i])
    df[i] = std_scaler.fit_transform(arr.reshape(len(arr),1))
    #To use the same scaler which was used in preprocessing the train data 
    att[i] = std_scaler.transform(x.reshape(len(x),1))
  return att
# calling the normalization() function
test = normalization(data.copy(),att.copy(),numeric_col)

In [10]:
att.head(5)

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
1296087,177144,1546,53966,2293,444,6,79,42664070,5,6,...,20,0.0,0.0000,0.0,0.0,0.0,0.000,0.0,0.0,Infiltration
1296088,177144,1546,53966,2293,444,6,79,2030,2,0,...,20,0.0,0.0000,0.0,0.0,0.0,0.000,0.0,0.0,Infiltration
1296089,177145,1546,54119,2293,444,6,88,119991834,5523,5525,...,20,997842.0,831471.3076,1585781.0,409903.0,8143987.5,1821773.648,9432176.0,6855799.0,Infiltration
1296090,177146,1546,54122,2293,444,6,90,5005212,15,15,...,28,0.0,0.0000,0.0,0.0,0.0,0.000,0.0,0.0,Infiltration
1296091,177145,1546,54119,2293,444,6,90,119973135,4376,4374,...,20,0.0,0.0000,0.0,0.0,0.0,0.000,0.0,0.0,Infiltration


# **One-hot-encoding**

In [11]:
att.Label = 'unknown'

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [12]:
att.head()

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
1296087,177144,1546,53966,2293,444,6,79,42664070,5,6,...,20,0.0,0.0000,0.0,0.0,0.0,0.000,0.0,0.0,unknown
1296088,177144,1546,53966,2293,444,6,79,2030,2,0,...,20,0.0,0.0000,0.0,0.0,0.0,0.000,0.0,0.0,unknown
1296089,177145,1546,54119,2293,444,6,88,119991834,5523,5525,...,20,997842.0,831471.3076,1585781.0,409903.0,8143987.5,1821773.648,9432176.0,6855799.0,unknown
1296090,177146,1546,54122,2293,444,6,90,5005212,15,15,...,28,0.0,0.0000,0.0,0.0,0.0,0.000,0.0,0.0,unknown
1296091,177145,1546,54119,2293,444,6,90,119973135,4376,4374,...,20,0.0,0.0000,0.0,0.0,0.0,0.000,0.0,0.0,unknown


In [17]:
X = att.drop('Label' , axis=1)
X = X.to_numpy().reshape(-1, 83,1)

y = att.Label

In [18]:
type(y)

pandas.core.series.Series

In [19]:
y

1296087    unknown
1296088    unknown
1296089    unknown
1296090    unknown
1296091    unknown
            ...   
1851548    unknown
1851549    unknown
1851550    unknown
1851551    unknown
1851552    unknown
Name: Label, Length: 19205, dtype: object

In [20]:
y.shape , X.shape

((19205,), (19205, 83, 1))

# **Load Model and Predict**

*We are checking only on the unknown label data but later on we will be checking for all data as well such that how many true labels it give with the threshold decided*

In [21]:
from tensorflow import keras
model = keras.models.load_model('/content/drive/MyDrive/models/model_split_cnn_model.hdf5')

In [22]:
predicted = model.predict(X)

601/601 [==============================] - 8s 2ms/step


In [23]:
Threshold_values = [0.4 ,0.5 , 0.6 , 0.7 , 0.8 , 0.9 , 0.95]

In [27]:
type(predicted[0])

numpy.ndarray

In [28]:
max_probability = np.amax(predicted, axis = 1)

In [29]:
max_probability.shape , type(max_probability)

((19205,), numpy.ndarray)

In [40]:
for threshold in Threshold_values:
  noofunknown = 0
  #for each threshold value we will check out the accuracy of the model
  print('USING THRESHOLD VALUE ',threshold)
  for prob in max_probability:
    if prob<threshold:
      ##add unknown count
      noofunknown += 1
  print('TOTAL NO OF SAMPLES ',len(max_probability))
  print('TOTAL NO OF UNKNOWN PREDICTED ',noofunknown)
  Accuracy = noofunknown/len(max_probability)
  print('Accuracy is ',Accuracy)

USING THRESHOLD VALUE  0.4
TOTAL NO OF SAMPLES  19205
TOTAL NO OF UNKNOWN PREDICTED  5478
Accuracy is  0.2852382192137464
USING THRESHOLD VALUE  0.5
TOTAL NO OF SAMPLES  19205
TOTAL NO OF UNKNOWN PREDICTED  5478
Accuracy is  0.2852382192137464
USING THRESHOLD VALUE  0.6
TOTAL NO OF SAMPLES  19205
TOTAL NO OF UNKNOWN PREDICTED  5478
Accuracy is  0.2852382192137464
USING THRESHOLD VALUE  0.7
TOTAL NO OF SAMPLES  19205
TOTAL NO OF UNKNOWN PREDICTED  5478
Accuracy is  0.2852382192137464
USING THRESHOLD VALUE  0.8
TOTAL NO OF SAMPLES  19205
TOTAL NO OF UNKNOWN PREDICTED  5478
Accuracy is  0.2852382192137464
USING THRESHOLD VALUE  0.9
TOTAL NO OF SAMPLES  19205
TOTAL NO OF UNKNOWN PREDICTED  5478
Accuracy is  0.2852382192137464
USING THRESHOLD VALUE  0.95
TOTAL NO OF SAMPLES  19205
TOTAL NO OF UNKNOWN PREDICTED  5478
Accuracy is  0.2852382192137464


In [30]:
data = pd.DataFrame(max_probability)

In [31]:
data.head()

,0
0,0.0
1,1.0
2,1.0
3,1.0
4,1.0


array([0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.,
       0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1.

In [36]:
len(predicted)

19205

In [38]:
predicted[0]

array([0., 0., 0., 0., 0., 0.], dtype=float32)